In [1]:
import pandas as pd
import numpy as np

Pассчитайте винтажное время стояния в разных аэропортах по отдельным аэропортам. Сравните винтажное распределение стояния в аэропорту, когда водитель получил заказ и когда уехал без заказа (skytaxi.airport_visit, уехал с заказом - left_w_order = 1)

Скрипт, использованный для выгрузки таблицы:
```
select * from 
skytaxi.airport_visit
```

In [4]:
data = pd.read_csv('airport_vintage.csv')
data[['time_came', 'time_left']] = data[['time_came', 'time_left']].apply(pd.to_datetime)
data;

Добавим колонку, значения в которой будут соответствовать времени стояния в аэропорту, а также колонку с часом прибытия (для определения времени суток)

In [40]:
data['await_time'] = data['time_left'].sub(data['time_came'])
data['await_time'] = data['await_time'].apply(lambda x: int((x.seconds)/60))
data['arrive_hour'] = data['time_came'].apply(lambda x: x.hour)
data.sort_values('time_came', inplace=True)

Забинируем время прибытия в аэропорт

In [53]:
bins_hour = [-1, 6, 12, 24]
labels = ['night', 'morning', 'day']
data['day_part'] = pd.cut(data.arrive_hour, bins=bins_hour, labels=labels)

In [54]:
data

,id_driver,id_port,time_came,time_left,left_w_order,await_time,arrive_hour,day_part
303,6244,Кольцово,2021-06-01 00:04:00,2021-06-01 04:14:09,0,250,0,night
392,6315,Внуково,2021-06-01 00:52:00,2021-06-01 05:35:23,1,283,0,night
362,6291,Пулково,2021-06-01 02:38:00,2021-06-01 06:00:09,0,202,2,night
578,6471,Домодедово,2021-06-01 02:55:00,2021-06-01 05:25:27,1,150,2,night
276,6220,Пулково,2021-06-01 03:06:00,2021-06-01 11:24:27,0,498,3,night
...,...,...,...,...,...,...,...,...
18,6017,Пулково,2021-06-30 22:01:00,2021-07-01 01:38:51,0,217,22,day
868,6706,Домодедово,2021-06-30 22:23:00,2021-06-30 23:26:41,1,63,22,day
76,6060,Домодедово,2021-06-30 22:41:00,2021-06-30 22:53:55,1,12,22,day
1013,6824,Пулково,2021-06-30 22:58:00,2021-07-01 07:48:46,0,530,22,day


In [55]:
driver_count = data[['id_driver', 'id_port', 'day_part']].groupby(['id_port', 'day_part']).count()
driver_count.rename(columns={'id_driver':'driver_cnt'}, inplace=True)
total_waiting_time = data[['await_time', 'id_port', 'day_part']].groupby(['id_port', 'day_part']).sum()

In [56]:
driver_count

driver_cnt
id_port       day_part            
Аэропорт Сочи night             21
              morning           22
              day               30
Внуково       night             53
              morning           33
              day               71
Домодедово    night            101
              morning           81
              day              146
Кольцово      night             32
              morning           29
              day               31
Пулково       night            124
              morning           67
              day              156
Шереметьево   night             70
              morning           56
              day               99

In [72]:
total_waiting_time

await_time
id_port       day_part            
Аэропорт Сочи night           6099
              morning         6603
              day             8117
Внуково       night          17430
              morning         9308
              day            17403
Домодедово    night          29780
              morning        22331
              day            39596
Кольцово      night           8144
              morning         7935
              day             8441
Пулково       night          36678
              morning        18700
              day            38467
Шереметьево   night          19829
              morning        14993
              day            27999

In [74]:
mean_waiting_time = total_waiting_time
for i in data['id_port'].unique():
    for j in data['day_part'].unique():
        mean_waiting_time.loc[(i, j)]['await_time'] /= driver_count.loc[(i,j)]['driver_cnt']


In [75]:
mean_waiting_time

await_time
id_port       day_part            
Аэропорт Сочи night            290
              morning          300
              day              270
Внуково       night            328
              morning          282
              day              245
Домодедово    night            294
              morning          275
              day              271
Кольцово      night            254
              morning          273
              day              272
Пулково       night            295
              morning          279
              day              246
Шереметьево   night            283
              morning          267
              day              282

Сравним теперь винтажное время стояния водителей, уехавших с заказом со временем стояния водителей, уехавших без заказа

In [76]:
driver_count_compare = data[['id_driver', 'id_port', 'day_part', 'left_w_order']].groupby(['id_port', 'day_part', 'left_w_order']).count()
driver_count_compare.rename(columns={'id_driver':'driver_cnt'}, inplace=True)
total_waiting_time_compare = data[['await_time', 'id_port', 'day_part', 'left_w_order']].groupby(['id_port', 'day_part', 'left_w_order']).sum()

In [78]:
mean_waiting_time_compare = total_waiting_time_compare
for i in data['id_port'].unique():
    for j in data['day_part'].unique():
        for k in data['left_w_order'].unique():    
            mean_waiting_time_compare.loc[(i, j, k)]['await_time'] /= driver_count_compare.loc[(i,j, k)]['driver_cnt']

Запишем в список `diffs` разницу между средним временем стояния водителей, уехавших с заказом и средним временем стояния водителей, уехавших без заказа.

In [102]:
times = np.array(mean_waiting_time_compare['await_time'])
diffs = [times[i+1] - times[i] for i in range(0, len(times), 2)]

In [109]:
pd.concat([mean_waiting_time, pd.Series(diffs, index=mean_waiting_time.index, name='time_diff')], axis=1)

await_time  time_diff
id_port       day_part                       
Аэропорт Сочи night            290        165
              morning          300        -26
              day              270        -52
Внуково       night            328         82
              morning          282         57
              day              245         -1
Домодедово    night            294         73
              morning          275        -21
              day              271        -12
Кольцово      night            254         22
              morning          273         36
              day              272        -83
Пулково       night            295         41
              morning          279         22
              day              246         30
Шереметьево   night            283         47
              morning          267         44
              day              282       -121